In [1]:
import cv2
from PIL import Image
import os
import numpy as np
import pandas as pd
from hashlib import md5
import ast
from scipy import ndimage

df = pd.read_csv('segmented.csv')

df = df.dropna()

old_df = None
if os.path.exists('validated.csv'):
    old_df = pd.read_csv('validated.csv')
    
# remove if already validated
if old_df is not None:
    df = df[~df['mask'].isin(old_df['mask'])]

df

,image,x,y,mask
1,/home/jack/Mounts/DiskOne/kona_coffee/splits/1...,308,147,/home/jack/Mounts/DiskOne/kona_coffee/masks/10...
2,/home/jack/Mounts/DiskOne/kona_coffee/splits/1...,273,222,/home/jack/Mounts/DiskOne/kona_coffee/masks/10...
15,/home/jack/Mounts/DiskOne/kona_coffee/splits/1...,76,34,/home/jack/Mounts/DiskOne/kona_coffee/masks/10...
79,/home/jack/Mounts/DiskOne/kona_coffee/splits/1...,226,296,/home/jack/Mounts/DiskOne/kona_coffee/masks/10...
80,/home/jack/Mounts/DiskOne/kona_coffee/splits/1...,268,221,/home/jack/Mounts/DiskOne/kona_coffee/masks/10...
...,...,...,...,...
2611,/home/jack/Mounts/DiskOne/kona_coffee/augmente...,61,394,/home/jack/Mounts/DiskOne/kona_coffee/masks/10...
2612,/home/jack/Mounts/DiskOne/kona_coffee/augmente...,56,319,/home/jack/Mounts/DiskOne/kona_coffee/masks/10...
2613,/home/jack/Mounts/DiskOne/kona_coffee/augmente...,364,104,/home/jack/Mounts/DiskOne/kona_coffee/masks/10...
2614,/home/jack/Mounts/DiskOne/kona_coffee/augmente...,35,378,/home/jack/Mounts/DiskOne/kona_coffee/masks/10...


In [2]:
cv2.namedWindow('image')

def get_display_image(row):
    cv2image = cv2.imread(row.image)

    mask = np.load(row['mask'])
    mask = mask.astype(np.uint8)
    
    # apply the mask to the image
    mask_image = cv2.bitwise_and(cv2image, cv2image, mask=mask)
    
    # crop the image to the mask
    mask_image = mask_image[np.ix_(mask.any(1),mask.any(0))]
    
    # create alpha mask from mask_image
    alpha_mask = np.zeros(mask_image.shape[:2], dtype=np.uint8)
    
    # apply the mask to the image
    alpha_mask[mask_image[:,:,0] > 0] = 255
    
    return mask_image


image_index = 0
data = []

show_image = get_display_image(df.iloc[image_index])

while True:
    
    if show_image is None or show_image.shape[0] <= 0 or show_image.shape[1] <= 0:
        print(f'invalid mask {df.iloc[image_index]["mask"]}')
        image_index += 1
        if image_index >= len(df):
            break
        show_image = get_display_image(df.iloc[image_index])
        continue
        
    cv2.imshow("image", show_image)
    k = cv2.waitKey(50) & 0xFF
    
    # if escape key, exit
    if k == 27:
        break

    elif k == ord('n'):
        row = df.iloc[image_index]
        data.append([row.image, row.x, row.y, row['mask'], False])
        image_index += 1
        if image_index >= len(df):
            break
        
        show_image = get_display_image(df.iloc[image_index])
        
    elif k == ord('y'):
        row = df.iloc[image_index]
        data.append([row.image, row.x, row.y, row['mask'], True])
        image_index += 1
        if image_index >= len(df):
            break
        
        show_image = get_display_image(df.iloc[image_index])


# close all open windows
cv2.destroyAllWindows()

qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/jack/.local/share/virtualenvs/datadev-5ox7fytP/lib/python3.10/site-packages/cv2/qt/plugins"


invalid mask /home/jack/Mounts/DiskOne/kona_coffee/masks/100_0045_0_2400.308.147.mask.npy
invalid mask /home/jack/Mounts/DiskOne/kona_coffee/masks/100_0045_0_2400.273.222.mask.npy
invalid mask /home/jack/Mounts/DiskOne/kona_coffee/masks/100_0060_3200_3200.76.34.mask.npy
invalid mask /home/jack/Mounts/DiskOne/kona_coffee/masks/100_0029_2400_800.226.296.mask.npy
invalid mask /home/jack/Mounts/DiskOne/kona_coffee/masks/100_0029_2400_800.268.221.mask.npy
invalid mask /home/jack/Mounts/DiskOne/kona_coffee/masks/100_0029_2400_800.301.305.mask.npy
invalid mask /home/jack/Mounts/DiskOne/kona_coffee/masks/100_0059_4000_400.191.90.mask.npy
invalid mask /home/jack/Mounts/DiskOne/kona_coffee/masks/100_0059_4000_400.86.232.mask.npy
invalid mask /home/jack/Mounts/DiskOne/kona_coffee/masks/100_0059_4000_400.158.214.mask.npy
invalid mask /home/jack/Mounts/DiskOne/kona_coffee/masks/100_0059_4000_400.107.297.mask.npy
invalid mask /home/jack/Mounts/DiskOne/kona_coffee/masks/100_0045_0_1600.316.175.mask.n

In [3]:
df = pd.DataFrame(data, columns=['image', 'x', 'y', 'mask', 'valid'])

if old_df is not None:
    df = pd.concat([old_df, df])

df.to_csv('validated.csv', index=False)

df

,image,x,y,mask,valid
0,/home/jack/Mounts/DiskOne/kona_coffee/splits/1...,299,58,/home/jack/Mounts/DiskOne/kona_coffee/masks/10...,True
1,/home/jack/Mounts/DiskOne/kona_coffee/splits/1...,324,300,/home/jack/Mounts/DiskOne/kona_coffee/masks/10...,True
2,/home/jack/Mounts/DiskOne/kona_coffee/splits/1...,272,321,/home/jack/Mounts/DiskOne/kona_coffee/masks/10...,True
3,/home/jack/Mounts/DiskOne/kona_coffee/splits/1...,364,288,/home/jack/Mounts/DiskOne/kona_coffee/masks/10...,True
4,/home/jack/Mounts/DiskOne/kona_coffee/splits/1...,228,197,/home/jack/Mounts/DiskOne/kona_coffee/masks/10...,False
...,...,...,...,...,...
309,/home/jack/Mounts/DiskOne/kona_coffee/augmente...,266,105,/home/jack/Mounts/DiskOne/kona_coffee/masks/10...,True
310,/home/jack/Mounts/DiskOne/kona_coffee/augmente...,262,262,/home/jack/Mounts/DiskOne/kona_coffee/masks/10...,True
311,/home/jack/Mounts/DiskOne/kona_coffee/augmente...,224,208,/home/jack/Mounts/DiskOne/kona_coffee/masks/10...,True
312,/home/jack/Mounts/DiskOne/kona_coffee/augmente...,193,156,/home/jack/Mounts/DiskOne/kona_coffee/masks/10...,False
